In [1]:
import pandas as pd
from pandas_profiling import ProfileReport


<ipython-input-1-98f09099f855>:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
data = pd.read_csv(r'C:\Users\Never\Downloads\Customer-Churn-Records.csv')

Getting report of data info by pandas profiling

In [3]:
profile = data.profile_report()
profile.to_file(r'C:\Users\Never\Downloads\profile_report.html')



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]